In [6]:
import numpy as np
import qupid.utils as spu
from qupid.qupid import QuPID
import warnings
import time
warnings.filterwarnings('ignore')
dataset = "ORBIT5K"

### Data Generation
In this section, we generate synthetic data representing different orbits. This involves creating a series of points that simulate the path of an object in an orbital motion, based on specified parameters.


In [ ]:
spu.compute_persistence(dataset)

### Data Processing
Here, we prepare the generated data for analysis by computing persistent diagrams for $H_0$ and $H_1$ homology groups for each sample. 


In [ ]:
diagrams_dict, labels, n_data = spu.get_data(dataset)

### Vectorization timings


QUPID

In [ ]:
parameter_configs = [
    {"function": "id", "wave": " "},
    {"function": "fft", "wave": " "},
    {"function": "wvt", "wave": "coif2"},
    {"function": "wvt", "wave": "db2"}
]

# Loop through each parameter configuration
for params in parameter_configs:
    samplesH0, samplesH1 = spu.process_diagrams(diagrams_dict)

    max_point0, max_point1 = spu.max_measures({"H0": samplesH0}), spu.max_measures({"H1": samplesH1})

    modelH0 = QuPID(**params, resolution=(1, 32), global_max=max_point0, global_min = (0, 0), alpha=(0, 1e3))
    modelH1 = QuPID(**params, resolution=(32, 32), global_max=max_point1, global_min = (0, 0), alpha=(1e3, 1e3))

    start_time = time.time()
    data = []
    for i in range(n_data):
        modelH0.fit([samplesH0[i]])
        modelH1.fit([samplesH1[i]])
        data.append(np.concatenate((modelH0.transform(samplesH0[i]), modelH1.transform(samplesH1[i]))))
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    print(f"Elapsed time for {params}': {elapsed_time:.4f} seconds")

PI

In [ ]:
from gudhi.representations import PersistenceImage

samplesH0, samplesH1 = spu.process_diagrams(diagrams_dict)
pi= PersistenceImage(resolution = (32, 32))

data = []
start_time = time.time()
for i in range(n_data):
    vec = pi.fit_transform([samplesH0[i], samplesH1[i]])
    data.append(vec.flatten())
end_time = time.time()
elapsed_time = end_time - start_time

PF-K 

In [ ]:
from gudhi.representations import PersistenceFisherKernel

samplesH0, samplesH1 = spu.process_diagrams(diagrams_dict)
pfk_model = PersistenceFisherKernel()

data = []
start_time = time.time()
for i in range(n_data):
    vec = pfk_model.fit_transform([samplesH0[i], samplesH1[i]])
    data.append(vec.flatten())
end_time = time.time()
elapsed_time = end_time - start_time
